In [4]:
!pip install pynput

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.8/739.8 kB 24.1 MB/s eta 0:00:00
  Attempting uninstall: pyobjc-core
    Found existing installation: pyobjc-core 10.1
    Uninstalling pyobjc-core-10.1:
      Successfully uninstalled pyobjc-core-10.1
  Attempting uninstall: pyobjc-framework-Cocoa
    Found existing installation: pyobjc-framework-Cocoa 10.1
    Uninstalling pyobjc-framework-Cocoa-10.1:
      Successfully uninstalled pyobjc-framework-Cocoa-10.1


In [1]:
import mediapipe as mp
import cv2
from pynput.keyboard import Key, Controller

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
keyboard = Controller()

In [3]:
cap = cv2.VideoCapture(0)
cap.set(3, 720) 
cap.set(4, 540)
pose = ""
status = 0

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:

    while True:
        success, frame = cap.read()
        frame = cv2.flip(frame,1)
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = holistic.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        height, width, _ = img.shape

        try:
            right_hand = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x*width,
                         results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y*width)

            line_x1 = width//3
            line_x2 = 2*(width//3)
            line_y1 = height//3
            line_y2 = 2*(height//3)

            if right_hand[0]>line_x2 and right_hand[1]<line_y1:
                pose = "Start"
                status = 1
                keyboard.press(Key.space)
                keyboard.release(Key.space)

            elif right_hand[0]>line_x2 and right_hand[1]>line_y1 and right_hand[1]<line_y2 and status==1:
                pose="Right"
                keyboard.press(Key.right)
                keyboard.release(Key.right)

            elif right_hand[0]<line_x1 and right_hand[1]>line_y1 and right_hand[1]<line_y2 and status==1:
                pose="Left"
                keyboard.press(Key.left)
                keyboard.release(Key.left)

            elif right_hand[1]<line_y1 and status==1:
                pose="Jump"
                keyboard.press(Key.up)
                keyboard.release(Key.up)

            elif right_hand[1]>line_y2 and status==1:
                pose="Slide"
                keyboard.press(Key.down)
                keyboard.release(Key.down)

            elif status == 0:
                pose = "Please start the Game"

            else:
                pose = "Run"

        except:
            pass

        cv2.putText(img,pose,(10,30), cv2.FONT_HERSHEY_PLAIN, 2,(255,255,0),3)

        cv2.line(img,(width//3,0),(width//3,height),(255,255,255),2)
        cv2.line(img,(2*(width//3),0),(2*(width//3),height),(255,255,255),2)
        cv2.line(img,(0,height//3),(width,height//3),(255,255,255),2)
        cv2.line(img,(0,2*(height//3)),(width,2*(height//3)),(255,255,255),2)

        mp_drawing.draw_landmarks(img,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(180,105,255),thickness=5,circle_radius=8),
                                 mp_drawing.DrawingSpec(color=(255,255,255),thickness=10,circle_radius=10))
        
        cv2.imshow('Image', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1748661148.934230   63884 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1748661149.036282   64302 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748661149.059325   64297 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748661149.062308   64300 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748661149.062363   64295 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748661149.062531   64302 inference_feedback_manager.cc:114] Feedback manager requires a mod